In [1]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torchsampler import ImbalancedDatasetSampler
import matplotlib.pyplot as plt
import os
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from functools import reduce
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix
import pandas as pd
import sklearn 
from torch.autograd import Variable

In [2]:
import loaders as l
import models as m
import train_eval as te

In [3]:
import cv2

In [4]:

from ImageClassification import helpers as h
import argparse
import time

In [5]:
saved_model_path='C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/firstModel2'
net = m.BaseNet_v3()
net.load_state_dict(torch.load(saved_model_path))

<All keys matched successfully>

In [6]:
cv2.destroyAllWindows() 
# load the image and define the window width and height
from PIL import Image
from matplotlib import cm
image = cv2.imread("C:/Users/33783/Desktop/start_deep/start_deep/Projet_ML/test.jpg")
(winW, winH) = (36, 36)
transform = transforms.Compose(
    [transforms.Grayscale(), 
     transforms.ToTensor(), 
     transforms.Normalize(mean=(0,),std=(1,))])
face_detected=0
layer=0
color_face=(255,0,0)
color_noFace=(0, 255, 0)
sleep=0.0015
detected_faces=[]
sc=1.2

# loop over the image pyramid
for resized in h.pyramid(image, scale=sc):
    
    # loop over the sliding window for each layer of the pyramid
    
    
    for (x, y, window) in h.sliding_window(resized, stepSize=10, windowSize=(winW, winH)):
        # if the window does not meet our desired window size, ignore it
        if window.shape[0] != winH or window.shape[1] != winW:
            continue

        # THIS IS WHERE YOU WOULD PROCESS YOUR WINDOW, SUCH AS APPLYING A
        # MACHINE LEARNING CLASSIFIER TO CLASSIFY THE CONTENTS OF THE
        # WINDOW
        color=color_noFace
        sleep=0.005

        im = Image.fromarray(window.astype('uint8'), 'RGB')
        im=transform(im)
        im = im.unsqueeze(0)
        outputs=net(im)
        prob=torch.nn.functional.softmax(outputs, dim=1).detach()[0][1].numpy()
        #print(prob)
        y_pred_softmax = torch.log_softmax(outputs, dim = 1)
        _, y_pred_tags = torch.max(y_pred_softmax, dim = 1)
        prediction=np.array(y_pred_tags)
 
        new_face=[]
        if prediction==1 and prob>0.9:
            face_detected+=1
            new_face.append(layer)
            new_face.append(x*pow(sc,layer))
            new_face.append(y*pow(sc,layer))
            new_face.append((prob))
            detected_faces.append(new_face)
            print(new_face)
            color=color_face
            sleep=0.5
        
            
        #draw the window
        clone = resized.copy()
        cv2.rectangle(clone, (x, y), (x + winW, y + winH), color, 2)
        cv2.imshow("Window", clone)
        cv2.waitKey(1)
        time.sleep(sleep)
        
    layer+=1
        
print(face_detected)
cv2.waitKey(1)
cv2.destroyAllWindows() 



[0, 50.0, 10.0, array(0.9180417, dtype=float32)]
[0, 220.0, 20.0, array(0.9962572, dtype=float32)]
[0, 250.0, 20.0, array(0.99993527, dtype=float32)]
[0, 40.0, 40.0, array(0.9840551, dtype=float32)]
[0, 320.0, 70.0, array(0.9983094, dtype=float32)]
[0, 130.0, 90.0, array(0.925615, dtype=float32)]
[0, 140.0, 90.0, array(0.99261343, dtype=float32)]
[0, 170.0, 90.0, array(0.9969426, dtype=float32)]
[0, 310.0, 170.0, array(0.9963611, dtype=float32)]
[0, 240.0, 190.0, array(0.9159373, dtype=float32)]
[1, 36.0, 36.0, array(0.9722863, dtype=float32)]
[1, 204.0, 36.0, array(0.9870494, dtype=float32)]
[1, 24.0, 120.0, array(0.997775, dtype=float32)]
[1, 240.0, 180.0, array(0.9328685, dtype=float32)]
[1, 252.0, 252.0, array(0.999995, dtype=float32)]
[2, 172.79999999999998, 86.39999999999999, array(0.975999, dtype=float32)]
[2, 86.39999999999999, 144.0, array(0.96541226, dtype=float32)]
[2, 100.8, 144.0, array(0.93168277, dtype=float32)]
[2, 288.0, 216.0, array(0.9931525, dtype=float32)]
[3, 241.

In [10]:
cv2.destroyAllWindows() 

In [8]:
cv2.destroyAllWindows() 
nb_layers=detected_faces[-1][0]


for i in detected_faces : 


    x=i[1]+(winW*pow(1.2,i[0])/2)
    y=i[2]+(winH*pow(1.2,i[0])/2)

    #cv2.circle(image, (int(x), int(y)), 1,color_face, 3)

    cv2.rectangle(image, (int(i[1]), int(i[2])), (int(i[1]) +int(winW*pow(sc,i[0])), int(i[2]) + int(winH*pow(sc,i[0]))), color_face, 2)

cv2.imshow("Layer", image)

cv2.waitKey(1)
#cv2.destroyAllWindows() 


-1

In [9]:
#cv2.destroyAllWindows() 